In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, pi
from numpy import array, arange, zeros_like
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score

# Double Pendulum Parameters
g = 9.81
L1 = 1
L2 = 1
m1 = 1.0
m2 = 1.0

# Function for Double Pendulum System
def f(r, t, L1, L2):
    theta1 = r[0]
    omega1 = r[1]
    theta2 = r[2]
    omega2 = r[3]

    ftheta1 = omega1
    fomega1 = (-g * (2 * m1 + m2) * sin(theta1) - m2 * g * sin(theta1 - 2 * theta2) - 2 * sin(theta1 - theta2) * m2 *
               (omega2**2 * L2 + omega1**2 * L1 * cos(theta1 - theta2))) / (L1 * (2 * m1 + m2 - m2 * cos(2 * theta1 - 2 * theta2)))

    ftheta2 = omega2
    fomega2 = (2 * sin(theta1 - theta2) * (omega1**2 * L1 * (m1 + m2) + g * (m1 + m2) * cos(theta1) + omega2**2 * L2 * m2 *
                                           cos(theta1 - theta2))) / (L2 * (2 * m1 + m2 - m2 * cos(2 * theta1 - 2 * theta2)))

    return array([ftheta1, fomega1, ftheta2, fomega2], float)

# Simulation Parameters
a = 0.0
b = 10
N = 2000
h = (b - a) / N

angles = angles = [[120, 0], [121, 1], [120, 2], [120,4], [120,6], [120,8], [120,10],[122, 0], [122, 2], [122,4], [122,6], [122,8], [122,10],[124, 0], [124, 2], [124,4], [124,6], [124,8], [124,10], [126, 0], [126, 2], [126,4], [126,6], [126,8], [126,10], [128, 0], [128, 2], [128,4], [128,6], [128,8], [128,10], [130, 0], [130, 2], [130,4], [130,6], [130,8], [130,10]]

for x in angles:
    tpoints = np.arange(a, b, h)
    theta1_points = np.zeros_like(tpoints)
    theta2_points = np.zeros_like(tpoints)

    q = np.array([x[0] * pi / 180, 0, x[1] * pi / 180, 0], float)

    for i, t in enumerate(tpoints):
        theta1_points[i] = q[0] * 180 / pi
        theta2_points[i] = q[2] * 180 / pi

        k1 = h * f(q, t, L1, L2)
        k2 = h * f(q + 0.5 * k1, t + 0.5 * h, L1, L2)
        k3 = h * f(q + 0.5 * k2, t + 0.5 * h, L1, L2)
        k4 = h * f(q + k3, t + h, L1, L2)
        q += (k1 + 2 * k2 + 2 * k3 + k4) / 6

    plt.plot(tpoints, theta1_points, label='Theta1')
    plt.plot(tpoints, theta2_points, label='Theta2')
    plt.title("Double Pendulum")
    plt.xlabel("t (seconds)")
    plt.ylabel("degrees")
    plt.legend()
    plt.show()

    data = np.stack((theta1_points, theta2_points), axis=1)
    np.save(f'pendulum_data_{str(x[0])}_{str(x[1])}.npy', data)

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dataset class
class QuadraticDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32).to(device)  # Shape (N, 3)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)  # Shape (N, 2)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

data = {}

for i in angles:
  loaded_data = np.load(f'pendulum_data_{str(i[0])}_{str(i[1])}.npy')
  scaler = MinMaxScaler()
  data_ = scaler.fit_transform(loaded_data)
  data[f'{str(i[0])}_{str(i[1])}'] = data_

print(data)

def create_io(data):
    x, x_1, x_2, y_1, y_2 = [], [], [], [], []
    for starting in data:
      starting_theta_1_degrees = int(starting.split("_")[0])
      starting_theta_2_degrees = int(starting.split("_")[1])

      starting_theta_1 = starting_theta_1_degrees * pi /180
      starting_theta_2 = starting_theta_2_degrees * pi /180

      angle_data = data[starting]
      for i in range(len(angle_data)):
          x.append(tpoints[i])
          x_1.append(starting_theta_1)
          x_2.append(starting_theta_2)
          y_1.append(angle_data[i][0])
          y_2.append(angle_data[i][1])
    return x, x_1, x_2, y_1, y_2

x, x_1, x_2, y_1, y_2 = create_io(data)

# Normalize data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

x_combined = np.vstack([x_1, x_2, x]).T
x_scaled = scaler_x.fit_transform(x_combined)

y_combined = np.vstack([y_1, y_2]).T
y_scaled = scaler_y.fit_transform(y_combined)

# Create dataset and dataloaders
dataset = QuadraticDataset(x_scaled, y_scaled)

test_loader = DataLoader(dataset, batch_size=10, shuffle=False)

x_dense_combined = np.vstack([x_1, x_2, x]).T
x_dense_scaled = scaler_x.transform(x_dense_combined)
x_dense_tensor = torch.tensor(x_dense_scaled, dtype=torch.float32).unsqueeze(1).to(device)
lstm_model.load_state_dict(torch.load('lstm_model.pth'))
lstm_model.eval()
with torch.no_grad():
    y_pred_scaled = vanillarnn(x_dense_tensor).cpu().numpy()



# Inverse transform predictions
y_pred_90_90 = scaler_y.inverse_transform(y_pred_scaled)

# RMSE and R^2 scores
rmse = np.sqrt(mean_squared_error(y_combined, y_pred_90_90))
r2 = r2_score(y_combined, y_pred_90_90)
print(f"Final RMSE: {rmse}")
print(f"Final R^2 Score: {r2}")

# Plotting results
plt.figure(figsize=(12, 6))
plt.plot(x, y_1, label='True y_1', color='blue', linestyle='--')
plt.plot(x, y_2, label='True y_2', color='green', linestyle='--')
plt.plot(x, y_pred_90_90[:, 0], label='Predicted y_1', color='red')
plt.plot(x, y_pred_90_90[:, 1], label='Predicted y_2', color='orange')
plt.xlabel('Time (s)')
plt.ylabel('Angle (degrees)')
plt.title('Comparison of True and Predicted Angles for Double Pendulum')
plt.legend()
plt.grid(True)
plt.show()

# Plotting training vs validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Generate predictions for 85, 85

angle = [120, 5]


tpoints = np.arange(a, b, h) # seconds, split into intervals of 100
theta1_points = np.zeros_like(tpoints) # what the first pendulm angle should be (empty)
theta2_points = np.zeros_like(tpoints) # what the second pendulm angle should be (empty)

q = np.array([angle[0]*pi/180, 0, angle[1]*pi/180, 0], float)

for i, t in enumerate(tpoints):
    theta1_points[i] = q[0] * 180 / pi # converted from theta (θ) to degrees (°)
    theta2_points[i] = q[2] * 180 / pi

    k1 = h * f(q, t, L1, L2)
    k2 = h * f(q + 0.5 * k1, t + 0.5 * h, L1, L2)
    k3 = h * f(q + 0.5 * k2, t + 0.5 * h, L1, L2)
    k4 = h * f(q + k3, t + h, L1, L2)
    q += (k1 + 2 * k2 + 2 * k3 + k4) / 6

x_1_unknown = [angle[0]*pi/180]*N
x_2_unknown = [angle[1]*pi/180]*N
x_unknown = tpoints.tolist()
scaler = MinMaxScaler()
y_1_2_unknown = scaler.fit_transform([[a, b] for a, b in zip(theta1_points, theta2_points)])
y_1_unknown, y_2_unknown = map(list, zip(*y_1_2_unknown))
y_combined = np.vstack([y_1_unknown, y_2_unknown]).T
y_scaled = scaler_y.fit_transform(y_combined)

x_dense_combined = np.vstack([x_1_unknown, x_2_unknown, x_unknown]).T
x_dense_scaled = scaler_x.transform(x_dense_combined)
x_dense_tensor = torch.tensor(x_dense_scaled, dtype=torch.float32).unsqueeze(1).to(device)  # Shape (500, 1, 2)

vanillarnn.eval()
with torch.no_grad():
    y_pred_scaled = vanillarnn(x_dense_tensor).cpu().numpy()

# Inverse transform predictions
y_pred_unknown = scaler_y.inverse_transform(y_pred_scaled)

# PLOTTING LOOKS A LITTLE OFF BECAUSE IT IS PLOTTING ALL ANGLES, NEEDS TO BE TURNED INTO NUMPY ARRAY AND SET TO NP.WHERE
# x_1 = initial angle 1 and x_2 is initial angle
# Calculate RMSE and R^2 score
rmse = np.sqrt(mean_squared_error(y_combined, y_pred_unknown))
r2 = r2_score(y_combined, y_pred_unknown)

print(f"RMSE on new angle: {rmse}")
print(f"R^2 Score on new angle: {r2}")

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(x_unknown, y_1_unknown, label='True y_1', color='blue', linestyle='--')
plt.plot(x_unknown, y_2_unknown, label='True y_2', color='green', linestyle='--')
plt.plot(x_unknown, y_pred_unknown[:, 0], label='Predicted y_1', color='red')
plt.plot(x_unknown, y_pred_unknown[:, 1], label='Predicted y_2', color='orange')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Comparison of True and Predicted Quadratic Curves')
plt.legend()
plt.grid(True)
plt.show()